<a href="https://colab.research.google.com/github/danielsaggau/IR_LDC/blob/main/hpo_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers
from datasets import load_dataset, load_metric
from transformers import (AutoModelForSequenceClassification, AutoTokenizer,
                          Trainer, TrainingArguments)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
dataset = load_dataset('glue', 'mrpc')
metric = load_metric('glue', 'mrpc')


In [ ]:
def encode(examples):
    outputs = tokenizer(
        examples['sentence1'], examples['sentence2'], truncation=True)
    return outputs

encoded_dataset = dataset.map(encode, batched=True)

In [4]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        'distilbert-base-uncased', return_dict=True)


In [5]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)
# Evaluate during training and a bit more often
# than the default to be able to prune bad trials early.
# Disabling tqdm is a matter of preference.

In [ ]:
!pip install 'ray[tune]'

In [10]:
training_args = TrainingArguments(
    "test", evaluation_strategy="steps", eval_steps=500, disable_tqdm=True)
trainer = Trainer(
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    model_init=model_init,
    compute_metrics=compute_metrics,
#    greater_is_better = True,
)
# Default objective is the sum of all metrics
# when metrics are provided, so we have to maximize it.
trainer.hyperparameter_search(
    direction="maximize", 
    backend="ray", 
    n_trials=10 # number of trials
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.22.2

(pid=397) Moving 0 files to the new cache system
== Status ==
Current time: 2022-10-06 15:13:09 (running for 00:00:00.57)
Memory usage on this node: 2.3/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 1.0/2 CPUs, 0/0 GPUs, 0.0/7.34 GiB heap, 0.0/3.67 GiB objects
Result logdir: /root/ray_results/_objective_2022-10-06_15-13-08
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+------------------------+----------+----------------+-----------------+--------------------+------------------------+----------+
| Trial name             | status   | loc            |   learning_rate |   num_train_epochs |   per_device_train_... |     seed |
|------------------------+----------+----------------+-----------------+--------------------+------------------------+----------|
| _objective_62ee6_00000 | RUNNING  | 172.28.0.2:397 |     5.61152e-06 |                  5 |                     64 |  8.15396 |
| _objective_62ee6_00001 | PENDING  |                |     1.56207e-05 |                  2

(_objective pid=397) Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
(_objective pid=397) - This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=397) - This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=397) Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased

== Status ==
Current time: 2022-10-06 15:13:24 (running for 00:00:15.53)
Memory usage on this node: 2.9/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 2.0/2 CPUs, 0/0 GPUs, 0.0/7.34 GiB heap, 0.0/3.67 GiB objects
Result logdir: /root/ray_results/_objective_2022-10-06_15-13-08
Number of trials: 10/10 (8 PENDING, 2 RUNNING)
+------------------------+----------+----------------+-----------------+--------------------+------------------------+----------+
| Trial name             | status   | loc            |   learning_rate |   num_train_epochs |   per_device_train_... |     seed |
|------------------------+----------+----------------+-----------------+--------------------+------------------------+----------|
| _objective_62ee6_00000 | RUNNING  | 172.28.0.2:397 |     5.61152e-06 |                  5 |                     64 |  8.15396 |
| _objective_62ee6_00001 | RUNNING  | 172.28.0.2:439 |     1.56207e-05 |                  2 |                     16 |  7.08379 |
| _objecti

(_objective pid=439) Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
(_objective pid=439) - This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=439) - This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=439) Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
| _objective_62ee6_00004 | PENDING  |                |     2.3102e-06  |                  5 |                      8 | 25.0818  |
| _objective_62ee6_00005 | PENDING  |                |     1.12076e-05 |                  4 |                     16 |  1.89943 |
| _objective_62ee6_00006 | PENDING  |                |     1.67381e-05 |                  2 |                     32 |  2.81996 |
| _objective_62ee6_00007 | PENDING  |                |     5.4041e-06  |                  3 |                     32 | 15.916   |
| _objective_62ee6_00008 | PENDING  |                |     1.53049e-05 |                  3 |                     64 | 34.5377  |
| _objective_62ee6_00009 | PENDING  |                |     7.96157e-06 |                  2 |                     32 | 38.0065  |
+------------------------+----------+----------------+-----------------+--------------------+------------------------+----------+


== Status ==
Current 

2022-10-06 16:01:49,633	WARNING tune.py:687 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 


== Status ==
Current time: 2022-10-06 16:01:50 (running for 00:48:41.40)
Memory usage on this node: 4.0/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 2.0/2 CPUs, 0/0 GPUs, 0.0/7.34 GiB heap, 0.0/3.67 GiB objects
Result logdir: /root/ray_results/_objective_2022-10-06_15-13-08
Number of trials: 10/10 (8 PENDING, 2 RUNNING)
+------------------------+----------+----------------+-----------------+--------------------+------------------------+----------+
| Trial name             | status   | loc            |   learning_rate |   num_train_epochs |   per_device_train_... |     seed |
|------------------------+----------+----------------+-----------------+--------------------+------------------------+----------|
| _objective_62ee6_00000 | RUNNING  | 172.28.0.2:397 |     5.61152e-06 |                  5 |                     64 |  8.15396 |
| _objective_62ee6_00001 | RUNNING  | 172.28.0.2:439 |     1.56207e-05 |                  2 |                     16 |  7.08379 |
| _objecti

KeyboardInterrupt: ignored

In [ ]:
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler

trainer = Trainer(
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    model_init=model_init,
    compute_metrics=compute_metrics,
)

best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="ray",
    # Choose among many libraries:
    # https://docs.ray.io/en/latest/tune/api_docs/suggestion.html
    search_alg=HyperOptSearch(metric="objective", mode="max"),
    # Choose among schedulers:
    # https://docs.ray.io/en/latest/tune/api_docs/schedulers.html
    scheduler=ASHAScheduler(metric="objective", mode="max"))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The module `ray.tune.suggest` has been moved to `ray.tune.search` and the old location will be deprecated soon. Please adjust your imports to point to the new location. Example: Do a global search and replace `ray.tune.suggest` with `ray.tune.search`.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The module `ray.tune.suggest.hyperopt` has been moved to `ray.tune.search.hyperopt` and the old location will be deprecated soon. Please adjust your imports to point to the new location. Example: Do a global search and replace `ray.tune.suggest.hyperopt` with `ray.tune.search.hyperopt`.
  """Entry point for launching an IPython kernel.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/config.json
Model config Di

== Status ==
Current time: 2022-10-06 16:02:13 (running for 00:00:04.44)
Memory usage on this node: 3.3/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: None
Resources requested: 1.0/2 CPUs, 0/0 GPUs, 0.0/7.34 GiB heap, 0.0/3.67 GiB objects
Result logdir: /root/ray_results/_objective_2022-10-06_16-02-08
Number of trials: 1/20 (1 RUNNING)
+---------------------+----------+-----------------+-----------------+--------------------+------------------------+---------+
| Trial name          | status   | loc             |   learning_rate |   num_train_epochs |   per_device_train_... |    seed |
|---------------------+----------+-----------------+-----------------+--------------------+------------------------+---------|
| _objective_3afed004 | RUNNING  | 172.28.0.2:1219 |     4.46483e-05 |                  5 |                      4 | 39.4722 |
+---------------------+----------+-----------------+-----------------+------

(_objective pid=1219) Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias']
(_objective pid=1219) - This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=1219) - This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=1219) Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-unc

== Status ==
Current time: 2022-10-06 16:02:22 (running for 00:00:14.04)
Memory usage on this node: 3.5/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: None
Resources requested: 2.0/2 CPUs, 0/0 GPUs, 0.0/7.34 GiB heap, 0.0/3.67 GiB objects
Result logdir: /root/ray_results/_objective_2022-10-06_16-02-08
Number of trials: 2/20 (2 RUNNING)
+---------------------+----------+-----------------+-----------------+--------------------+------------------------+----------+
| Trial name          | status   | loc             |   learning_rate |   num_train_epochs |   per_device_train_... |     seed |
|---------------------+----------+-----------------+-----------------+--------------------+------------------------+----------|
| _objective_3afed004 | RUNNING  | 172.28.0.2:1219 |     4.46483e-05 |                  5 |                      4 | 39.4722  |
| _objective_43587a0c | RUNNING  | 172.28.0.2:1255 |     1.73287e-06 |  

(_objective pid=1255) Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
(_objective pid=1255) - This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=1255) - This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=1255) Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-unc

== Status ==
Current time: 2022-10-06 16:02:36 (running for 00:00:27.65)
Memory usage on this node: 4.5/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: None
Resources requested: 2.0/2 CPUs, 0/0 GPUs, 0.0/7.34 GiB heap, 0.0/3.67 GiB objects
Result logdir: /root/ray_results/_objective_2022-10-06_16-02-08
Number of trials: 3/20 (1 PENDING, 2 RUNNING)
+---------------------+----------+-----------------+-----------------+--------------------+------------------------+----------+
| Trial name          | status   | loc             |   learning_rate |   num_train_epochs |   per_device_train_... |     seed |
|---------------------+----------+-----------------+-----------------+--------------------+------------------------+----------|
| _objective_3afed004 | RUNNING  | 172.28.0.2:1219 |     4.46483e-05 |                  5 |                      4 | 39.4722  |
| _objective_43587a0c | RUNNING  | 172.28.0.2:1255 |     1.73